### Import required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

### read oil price data

In [2]:
oil = pd.read_csv('./data/oil.csv')
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(value = 0)
# oil['date'] = pd.to_datetime(oil['date'])
oil = oil.sort_values(by=['date'])
oil.shape

(1218, 2)

### the oil data has missing dates for the weekends so we generate a time series to complete the data.
#### -> get the start and end dates from the price history 

In [3]:
start=oil.iloc[0]['date']
end=oil.iloc[oil.shape[0] - 1 ]['date']
print(start, end)

('2013-01-01', '2017-08-31')


### Create tiemseries with date range of oil data

In [4]:
oil_date_range = pd.date_range(start=start, end=end, freq='D')
oil_date_range = oil_date_range.strftime('%Y-%m-%d')

### Create records for missing dates

In [5]:
oil_rows = []

print(oil[oil['date'] == '2013-01-01']['dcoilwtico'],oil[oil['date'] == '2017-08-31']['dcoilwtico'])

for index in  range(oil_date_range.shape[0]):
    current_date = oil_date_range[index]
#     print(oil[oil.date == current_date].shape[0] == 0, oil[oil.date == current_date])
    if(oil[oil.date == current_date].shape[0] == 0):
        oil_rows.append({'date': current_date, 'dcoilwtico': 0.})
print(len(oil_rows))
missing_dates=pd.DataFrame(oil_rows)
missing_dates.shape

(0    0.0
Name: dcoilwtico, dtype: float64, 1217    47.26
Name: dcoilwtico, dtype: float64)
486


(486, 2)

### Concatenate missing records with historical data

In [6]:
oil = pd.concat([oil, missing_dates])
oil.head(10)

,date,dcoilwtico
0,2013-01-01,0.00
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


### Sort oil data by date

In [7]:
oil = oil.sort_values(by=['date'])
oil.head(10)

,date,dcoilwtico
0,2013-01-01,0.00
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
0,2013-01-05,0.00
1,2013-01-06,0.00
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81


In [ ]:
plt.plot(oil['date'], oil['dcoilwtico'], 'o')
# oil.round(2)

### Replace zeros in price data with the price from the previous day

In [8]:
#oil_with_ema = pd.read_csv('./data/oil_with_moving_averages.csv')
previous = 0
current_price = 0

oil_no_missing_prices = []
for index in range(oil.shape[0]):
    current_date = oil.iloc[index]['date']
    current_price = oil.iloc[index]['dcoilwtico']
    if current_price > 0:
        oil_no_missing_prices.append({'date': current_date, 'dcoilwtico': current_price})
        previous = current_price
    else:
        oil_no_missing_prices.append({'date': current_date, 'dcoilwtico': previous})

oil = pd.DataFrame(oil_no_missing_prices)
print(len(oil_no_missing_prices),oil.shape)
# previous = 0
# for index in range(oilTimeSeries.shape[0]):
#     current = oilTimeSeries.iloc[index]['dcoilwtico']
#     current_date = oilTimeSeries.iloc[index]['date']
#     print(oilTimeSeries.loc[current_date]['dcoilwtico'])
#     if current == 0:
#         print("Setting row: ", current_date, " to : ",previous)
#         oilTimeSeries.loc[current_date]['dcoilwtico'] = previous
#         print(oilTimeSeries.loc[current_date]['dcoilwtico'])
#     else:
#         previous = current

(1704, (1704, 2))


In [9]:
oil.head(10)

,date,dcoilwtico
0,2013-01-01,0.00
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-05,93.12
5,2013-01-06,93.12
6,2013-01-07,93.20
7,2013-01-08,93.21
8,2013-01-09,93.08
9,2013-01-10,93.81


### Nomalize data - calculate mean and standard deviation

In [13]:
oil.shape
#oil = oil[oil.dcoilwtico > 0]
standard_deviation = np.std(oil['dcoilwtico'])
mean = np.mean(oil['dcoilwtico'])
print(mean, standard_deviation)
oil['dcoilwtico'] = (oil['dcoilwtico'] - mean) / standard_deviation
oil.head()

(67.6619659624414, 25.665356494905303)


,date,dcoilwtico,10EMA,50EMA,200EMA
0,2013-01-01,-2.636315,0.000000,0.000000,0.000000
1,2013-01-02,0.992701,49.021053,47.040404,46.691866
2,2013-01-03,0.986078,65.238376,62.660590,62.198715
3,2013-01-04,0.991922,73.345856,70.507734,69.989829
4,2013-01-05,0.991922,78.174589,75.214739,74.664412


### Calculate the 10, 50 and 200 day Weighted EMAs

In [14]:
counter = 0
beta10 = 0.9
beta50 = 0.98
previous = 0
beta200 = 0.99478
previous10Value = 0
previous50Value = 0
previous200Value = 0
        
for index in range(oil.shape[0]):
#     print(index)
    counter += 1
    currentValue = oil.iloc[index]['dcoilwtico']
    sub10EMA = np.power(beta10 ,counter)
    sub50EMA = np.power(beta50 ,counter)
    sub200EMA = np.power(beta200 ,counter)
    correction10EMA = 1 - sub10EMA
    correction50EMA = 1 - sub50EMA
    correction200EMA = 1 - sub200EMA
    w10EMA = (beta10 * previous10Value) + ((1 - beta10) * currentValue)
    w50EMA = (beta50 * previous50Value) + ((1 - beta50) * currentValue)
    w200EMA = (beta200 * previous200Value) + ((1 - beta200) * currentValue)
    oil.loc[index, '10EMA'] =  w10EMA / correction10EMA
    oil.loc[index, '50EMA'] = w50EMA / correction50EMA
    oil.loc[index, '200EMA'] = w200EMA / correction200EMA
    previous10Value = w10EMA
    previous50Value = w50EMA
    previous200Value = w200EMA
    previous = currentValue

In [ ]:
# counter = 0
# beta10 = 0.9
# beta50 = 0.98
# beta200 = 0.99478
# previous10Value = 0
# previous50Value = 0
# previous200Value = 0
# correction10EMA = 0
# correction50EMA = 0
# correction200EMA = 0
# w10EMA = 0
# w50EMA = 0
# w200EMA = 0
        
# for index in range(oil.shape[0]):
# #     print(index)
#     counter += 1
#     currentValue = oil.iloc[index]['dcoilwtico']
#     if currentValue == 0:
#         print(oil.loc[index,'date'])
#         currentValue = previousValue
#         oil.loc[index, '10EMA'] =  w10EMA / correction10EMA
#         oil.loc[index, '50EMA'] = w50EMA / correction50EMA
#         oil.loc[index, '200EMA'] = w200EMA / correction200EMA 
#     else:
#         sub10EMA = np.power(beta10 ,counter)
#         sub50EMA = np.power(beta50 ,counter)
#         sub200EMA = np.power(beta200 ,counter)
#         correction10EMA = 1 - sub10EMA
#         correction50EMA = 1 - sub50EMA
#         correction200EMA = 1 - sub200EMA
#         w10EMA = (beta10 * previous10Value) + ((1 - beta10) * currentValue)
#         w50EMA = (beta50 * previous50Value) + ((1 - beta50) * currentValue)
#         w200EMA = (beta200 * previous200Value) + ((1 - beta200) * currentValue)
#         oil.loc[index, '10EMA'] =  w10EMA / correction10EMA
#         oil.loc[index, '50EMA'] = w50EMA / correction50EMA
#         oil.loc[index, '200EMA'] = w200EMA / correction200EMA
#     print(counter, ": currentValue : ",currentValue)
#     print(counter, ": previousValue : ",previousValue)
#     print(counter, ": corrected 10EMA : ",w10EMA , sub10EMA,  correction10EMA,": ", (w10EMA / correction10EMA))
#     print(counter, ": corrected 50EMA : ", w50EMA, sub50EMA , correction50EMA,": ", (w50EMA / correction50EMA))
#     print(counter, ": corrected 200EMA : ", w200EMA, sub200EMA , correction200EMA,": ", (w200EMA / correction200EMA))
#         previous10Value = w10EMA
#         previous50Value = w50EMA
#         previous200Value = w200EMA
#         previous = currentValue

In [16]:
oil.tail(20)
# oil.dtypes
# oil[oil.date == '2013-01-05']

,date,dcoilwtico,10EMA,50EMA,200EMA
1684,2017-08-12,-0.734530,-0.737186,-0.779565,-0.759642
1685,2017-08-13,-0.734530,-0.736921,-0.778664,-0.759511
1686,2017-08-14,-0.782065,-0.741435,-0.778732,-0.759629
1687,2017-08-15,-0.782844,-0.745576,-0.778814,-0.759750
1688,2017-08-16,-0.812845,-0.752303,-0.779495,-0.760027
1689,2017-08-17,-0.802325,-0.757305,-0.779952,-0.760248
1690,2017-08-18,-0.743102,-0.755885,-0.779215,-0.760159
1691,2017-08-19,-0.743102,-0.754606,-0.778492,-0.760070
1692,2017-08-20,-0.743102,-0.753456,-0.777784,-0.759981
1693,2017-08-21,-0.789857,-0.757096,-0.778026,-0.760137


### Convert date dtype to datetime and save clean data to hdf5 file

In [19]:
oil['date'] = pd.to_datetime(oil['date'])
oil.to_hdf('./data/oil_normalized_with_ema', 'oil_normalized_with_ema',mode='w', format='table')

In [20]:
oil = pd.read_hdf('./data/oil_normalized_with_ema')
oil.head(10)

,date,dcoilwtico,10EMA,50EMA,200EMA
0,2013-01-01,-2.636315,-2.636315,-2.636315,-2.636315
1,2013-01-02,0.992701,-0.726306,-0.803478,-0.817059
2,2013-01-03,0.986078,-0.094430,-0.194869,-0.212865
3,2013-01-04,0.991922,0.221462,0.110880,0.090701
4,2013-01-05,0.991922,0.409604,0.294279,0.272836
5,2013-01-06,0.991922,0.533882,0.416503,0.394258
6,2013-01-07,0.995039,0.622277,0.504244,0.481437
7,2013-01-08,0.995429,0.687796,0.570070,0.546869
8,2013-01-09,0.990364,0.737188,0.620631,0.597184
9,2013-01-10,1.018807,0.780426,0.664165,0.640346
